# IMPORTS

In [1]:
# Imports 3third party

import time
import requests
import pandas as pd
import json
import boto3
import io
import boto3
from datetime import datetime, timedelta
import pyarrow as pa
import pyarrow.parquet as pq
from dotenv import load_dotenv



In [2]:
# IMPORTS/variables custom 
import os
import sys
sys.path.append(os.path.abspath('.'))

load_dotenv()
PATH_DIR = os.getenv("path_variable")
PROFILE_NAME = os.getenv("profile_name")
# Get the absolute path
absolute_directory = os.path.abspath(PATH_DIR)

# Add the directory to sys.path
sys.path.append(absolute_directory)


# Imports Resources
import pandas as pd
from pandas import json_normalize
from io import StringIO
from datetime import datetime, timedelta
#from api_requests import api_requestClass, soap_requestClass
#from parquet_model import write_parquet_to_s3_raw
from log_manager import S3LogManager
#from week_days import get_previous_business_days
from util import control_sucess, infer_schema, convert_to_string
from infom_module import web_scrap_info_money, validate_data
from json_upload_local import write_json_to_s3
import time

# Import configs_dev in notebook_dir 
from config_dev import BASE_URL, REGION_NAME, \
    BUCKET, RAW_PREFIX, LOG_PREFIX, NAME_FUN


In [3]:
print("CHANGE ALL THE MOBILE VARIABLES IN config_dev.py FILE")
print(f"{BUCKET} \n{RAW_PREFIX}\n {BASE_URL}")


CHANGE ALL THE MOBILE VARIABLES IN config_dev.py FILE
aws-serveless-project-lambda-teste 
dev/raw/webcrap/date_ingestion=
 https://api.infomoney.com.br/fii/0/1000/36/desc/imovel/null?type=json


# _DEV_

In [ ]:
import json
import boto3
import pandas as pd

def write_json_to_s3(df, bucket_name, prefix, date_col):

    # Obtém a data de ingestão da coluna especificada
    ingestion_date = pd.to_datetime(df[date_col].max())

    # Define o caminho do arquivo no S3 com particionamento 
    s3_path = f"{prefix}{ingestion_date.strftime('%Y%m%d')}/file.json"

    # Converte o DataFrame para JSON
    json_buffer = io.BytesIO()
    df.to_json(json_buffer, orient='records')
    
    # Obtém o tamanho do buffer JSON
    json_buffer_size = json_buffer.tell()

    # Escreve o buffer JSON no S3
    s3_resource = boto3.resource("s3")
    s3_resource.Object(bucket_name, s3_path).put(Body=json_buffer.getvalue())

    return f"s3://{bucket_name}/{s3_path}", json_buffer_size

In [4]:
##  Dev/_Debugging old version_

# Develop your code here

def web_screp_info_money(url):
  """
  Requests of the web screp Infomoney.

 Return: json_file
  """

  try:
    response = requests.get(url)
    
    if response.status_code != 200:
      return False
    
    data = response.json()
    
    return data

  except:
    return False


json_data = web_screp_info_money(BASE_URL)


df = pd.json_normalize(json_data)



print("Aqui")

table = pa.Table.from_pandas(df)
parquet_buffer = io.BytesIO()
pq.write_table(table, parquet_buffer, compression='snappy')
# session = boto3.Session(profile_name=PROFILE_NAME)
# s3_resource = boto3.resource('s3')
# s3_resource.Object(BUCKET, RAW_PREFIX).put(Body=parquet_buffer.getvalue())

print("foi")

Aqui
foi


In [4]:
# Testing out new functions.

json_data = web_scrap_info_money(BASE_URL)
schema =infer_schema(json_data)
check_json = validate_data(json_data)
timestamp_SP = datetime.now() - timedelta(hours=3)
#write_json_to_s3(check_json, BUCKET, RAW_PREFIX, timestamp_SP, PROFILE_NAME)

In [5]:
import pprint

pprint.pprint(check_json)

[{'DateRef': '2024-05-16T03:00:00Z',
  'FII_PVPA': 0.0,
  'Id': '66470ee1c7d5ad14c0c9ba01',
  'book_value': 907.372242,
  'cnpj': '21.126.204/0001-43',
  'cotistas': 0.0,
  'data_de_referencia_do_informe_mensal': '2024-04-30T03:00:00Z',
  'data_ultimo_rendimento': '2024-05-15T03:00:00Z',
  'giro_21du_pct': 0.000160437111867,
  'market_cap': 966565641.8,
  'patrimonio_contabil': 567597632.42,
  'pb': 0.0,
  'preco_do_etf': 1550.73,
  'preco_do_etf_tolerancia_de_10dus': 0.0,
  'quant_cotas': 625540.0,
  'razao_social': 'FUNDO DE INVESTIMENTO IMOBILIÁRIO BRASIL VAREJO',
  'ret_12m': 4915.2975,
  'ret_mes': 0.0,
  'ret_semana': 0.0,
  'riskscore': None,
  'soma_proventos_12m': 222.55,
  'taxa_administracao': 0.1,
  'taxa_performance': '0',
  'ticker': 'BVAR11',
  'tipo_de_investimento': 'Imóvel',
  'tipo_de_lastro': 'Varejo',
  'tipo_de_renda': 'Rendimentos e/ou ganho de capital',
  'ultimo_rendimento': 5.56,
  'valor_cota': 0.0,
  'volume_medio_21du': 73.8442857143,
  'volume_total_negoci

### Lambda fun model below : Uncode it to dev

In [4]:

# V0 inicial structure for logging ect
def lambda_handler(event, context):
    inicio = time.perf_counter()
    control = "OK"
    timestamp_SP = datetime.now() - timedelta(hours=3)
    timestamp_SP_api_pull = timestamp_SP - timedelta(days=1)
    
    # loop df ... [!WARNING] if dont use it change in row 58
    #df_append = pd.DataFrame()

    # api class for requests
    json_data = web_scrap_info_money(BASE_URL)
    
    # --------------------Api requests wra wrangling
    # ------------------- Use Try and execept
 
    #----------------- execept + control_var for logs management
    try:
        schema =infer_schema(json_data)
        check_json = validate_data(json_data)

    #----------------- execept + control_var for logs management

    
    except Exception as e:
         control = f'Error msg: {e}'
         print(
             "Project: The request returned 0 items. Original message: {msg}".format(
                 msg=e
             )
         )
                        
    # -------------------------- usual final strucuture for uploading and logging the ingestion.
    
    sucess_var = control_sucess(control)

    file_s3= 0
    if control == "OK":
        file_s3, file_size = write_json_to_s3(check_json, BUCKET, RAW_PREFIX, timestamp_SP, PROFILE_NAME)
        print(f"Project: Json file saved to S3 at: \n {file_s3}")
        print(f"Project: Json file size: {file_size} bytes")
        
    fim = time.perf_counter()
    log_manager = S3LogManager(BUCKET, LOG_PREFIX, control)
    time_var = round((fim - inicio),2)
    log_manager.add_log_entry(NAME_FUN, sucess_var, time_var, control, file_size)

# ! lambda_handler('','') !


### TESTING / DEBUGING IT

In [5]:
import time
import logging

start_time = time.perf_counter()

try:
    lambda_handler(None, None)
    logging.basicConfig(filename = "scrapper.log", level=logging.INFO, format='%(asctime)s - %(message)s', datefmt = '%Y-%m-%d %H:%M:%S')
    end = time.perf_counter()
    time = end - start_time
    logging.info(f"{NAME_FUN} - success - time {time:.2f}.")

except Exception as e:
    logging.error(f"Error.{NAME_FUN}:{e}")

LIGHTHOUSE: Json file saved to S3 at: 
 s3://aws-serveless-project-lambda-teste/dev/raw/webcrap/date_ingestion=20240502/file.json
LIGHTHOUSE: Json file size: 135561 bytes


In [4]:
# zip -r external_libs/python.zip  .
# zip -g external_libs/python.zip typing_extensions.py .
# pip install pydantic -t external_libs

In [ ]:
# pydantic==2.7.1
# pydantic_core==2.18.2